In [1]:
import pandas as pd
import chardet
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, roc_auc_score, roc_curve, auc, f1_score
from sklearn.preprocessing import OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
import numpy as np
import math
import joypy
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, TomekLinks
from imblearn.combine import SMOTETomek, SMOTEENN
from collections import Counter
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate 
from sklearn.feature_selection import SelectFromModel
from datetime import datetime
import matplotlib.ticker as ticker
from matplotlib.legend_handler import HandlerLine2D
#import ray 


from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, precision_recall_curve, precision_recall_fscore_support


# py -m pip install matplotlib

In [2]:
df_private_MTPL = pd.read_csv('df_private.txt', sep="\t")

In [3]:
df_private_MTPL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118600 entries, 0 to 118599
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   PRODUCT                    118600 non-null  object 
 1   Duration                   118600 non-null  object 
 2   Vehicle_age                118600 non-null  float64
 3   Make                       118600 non-null  object 
 4   GWP                        118600 non-null  float64
 5   vehicle_type_group_mapped  118600 non-null  object 
 6   number_of_claims           118600 non-null  float64
 7   Channel                    118600 non-null  object 
 8   Churn                      118600 non-null  int64  
 9   Renewal                    118600 non-null  int64  
 10  Policy_Duration            118600 non-null  int64  
 11  Gender                     118600 non-null  object 
 12  Client_Age                 118600 non-null  float64
 13  City                       11

In [4]:
df_private_MTPL['Vehicle_age'] = pd.to_numeric(df_private_MTPL['Vehicle_age'], errors='coerce')
df_private_MTPL['GWP'] = pd.to_numeric(df_private_MTPL['GWP'], errors='coerce')
df_private_MTPL['number_of_claims'] = pd.to_numeric(df_private_MTPL['number_of_claims'], errors='coerce')
df_private_MTPL['Client_Age'] = pd.to_numeric(df_private_MTPL['Client_Age'], errors='coerce')
df_private_MTPL['Policy_Duration'] = pd.to_numeric(df_private_MTPL['Policy_Duration'], errors='coerce')
df_private_MTPL['Years_of_driving'] = pd.to_numeric(df_private_MTPL['Years_of_driving'], errors='coerce')

df_private_MTPL['PRODUCT'] = pd.Categorical(df_private_MTPL['PRODUCT'])
df_private_MTPL['Duration'] = pd.Categorical(df_private_MTPL['Duration'])
df_private_MTPL['Make'] = pd.Categorical(df_private_MTPL['Make'])
df_private_MTPL['vehicle_type_group_mapped'] = pd.Categorical(df_private_MTPL['vehicle_type_group_mapped'])
df_private_MTPL['City'] = pd.Categorical(df_private_MTPL['City'])
df_private_MTPL['Churn'] = pd.Categorical(df_private_MTPL['Churn'])
df_private_MTPL['Renewal'] = pd.Categorical(df_private_MTPL['Renewal'])
df_private_MTPL['Gender'] = pd.Categorical(df_private_MTPL['Gender'])
df_private_MTPL['Channel'] = pd.Categorical(df_private_MTPL['Channel'])


In [5]:
df_private_MTPL2 = df_private_MTPL

numerical_columns = ['Vehicle_age', 'GWP', 'number_of_claims', 'Policy_Duration', 'Client_Age', 'Years_of_driving']

scaler = MinMaxScaler()
df_private_MTPL2[numerical_columns] = scaler.fit_transform(df_private_MTPL2[numerical_columns])

In [6]:
df_private_MTPL3 = df_private_MTPL2

categorical_columns = ['PRODUCT', 'Duration', 'Make', 'vehicle_type_group_mapped', 'Gender', 'City', 'Channel']

encoder = OneHotEncoder(sparse_output=False, handle_unknown='error')

X_encoded = encoder.fit_transform(df_private_MTPL3[categorical_columns])

encoded_column_names = encoder.get_feature_names_out(categorical_columns)

df_encoded = pd.DataFrame(X_encoded, columns=encoded_column_names)

df_private_MTPL3 = pd.concat([df_private_MTPL3.drop(columns=categorical_columns), df_encoded], axis=1)

In [7]:
df_private_MTPL3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118600 entries, 0 to 118599
Data columns (total 86 columns):
 #   Column                                        Non-Null Count   Dtype   
---  ------                                        --------------   -----   
 0   Vehicle_age                                   118600 non-null  float64 
 1   GWP                                           118600 non-null  float64 
 2   number_of_claims                              118600 non-null  float64 
 3   Churn                                         118600 non-null  category
 4   Renewal                                       118600 non-null  category
 5   Policy_Duration                               118600 non-null  float64 
 6   Client_Age                                    118600 non-null  float64 
 7   Years_of_driving                              118600 non-null  float64 
 8   PRODUCT_MOD                                   118600 non-null  float64 
 9   PRODUCT_MTPL                         

In [21]:
df_private_MTPL3.to_csv('df_private_dummies.txt', sep='\t', index=False)

In [9]:
X = df_private_MTPL3.drop(columns=['Churn']) 
y = df_private_MTPL3['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

#X_train_ht, X_test_ht, y_train_ht, y_test_ht = train_test_split(X_train, y_train, stratify=y_train, test_size=0.7, random_state=42)

In [94]:
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

df = pd.concat([X_train, y_train], axis=1)

df.to_csv('train_data.txt', sep='\t', index=False)

In [95]:
X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)

df = pd.concat([X_test, y_test], axis=1)

df.to_csv('test_data.txt', sep='\t', index=False)

In [97]:
y_train.head(10)

Churn
93009     0
22770     0
64140     0
19924     0
91101     0
53068     0
93016     0
4079      0
6006      0
68892     0

In [98]:
class_counts_resampled = y_train.value_counts()

print("Class Counts after RandomUnderSampler:")
print(class_counts_resampled)

Class Counts after RandomUnderSampler:
Churn
0        70999
1         4899
Name: count, dtype: int64


In [10]:
enn =  EditedNearestNeighbours()
X_train_enn, y_train_enn = enn.fit_resample(X_train, y_train)

class_counts_resampled = y_train_enn.value_counts()

print("Class Counts after RandomUnderSampler:")
print(class_counts_resampled)

#X_train_ht_enn, X_test_ht_enn, y_train_ht_enn, y_test_ht_enn = train_test_split(X_train_enn,y_train_enn, stratify=y_train_enn, test_size=0.3, random_state=42)

# 0    59422
# 1     4832

Class Counts after RandomUnderSampler:
Churn
0    62808
1     7612
Name: count, dtype: int64


In [11]:
X_train_enn_df = pd.DataFrame(X_train_enn)
y_train_enn_df = pd.DataFrame(y_train_enn)

df = pd.concat([X_train_enn_df, y_train_enn_df], axis=1)

df.to_csv('train_data_enn.txt', sep='\t', index=False)

In [48]:
# df_enn = pd.read_csv('train_data_enn.txt', sep='\t')

# X_train_enn = df_enn.drop('Churn', axis=1)  
# y_train_enn = df_enn['Churn']              

In [11]:
sm =  SMOTE()
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

class_counts_resampled = y_train_sm.value_counts()

print("Class Counts after RandomUnderSampler:")
print(class_counts_resampled)

#X_train_ht_sm, X_test_ht_sm, y_train_ht_sm, y_test_ht_sm = train_test_split(X_train_sm,y_train_sm, stratify=y_train_sm, test_size=0.8, random_state=42)

# 0    75408
# 1    75408

Class Counts after RandomUnderSampler:
Churn
0    75408
1    75408
Name: count, dtype: int64


In [12]:
X_train_sm_df = pd.DataFrame(X_train_sm)
y_train_sm_df = pd.DataFrame(y_train_sm)

df = pd.concat([X_train_sm_df, y_train_sm_df], axis=1)

df.to_csv('train_data_sm.txt', sep='\t', index=False)

In [13]:
smt =  SMOTETomek()
X_train_smt, y_train_smt = smt.fit_resample(X_train, y_train)

class_counts_resampled = y_train_smt.value_counts()

print("Class Counts after RandomUnderSampler:")
print(class_counts_resampled)


#X_train_ht_smt, X_test_ht_smt, y_train_ht_smt, y_test_ht_smt = train_test_split(X_train_smt, y_train_smt, stratify=y_train_smt, test_size=0.7, random_state=42)

# 0    74637
# 1    74637

Class Counts after RandomUnderSampler:
Churn
0    74637
1    74637
Name: count, dtype: int64


In [49]:
df_sm = pd.read_csv('train_data_sm.txt', sep='\t')

X_train_sm = df_sm.drop('Churn', axis=1)  
y_train_sm = df_sm['Churn']              

In [14]:
X_train_smt_df = pd.DataFrame(X_train_smt)
y_train_smt_df = pd.DataFrame(y_train_smt)

df = pd.concat([X_train_smt_df, y_train_smt_df], axis=1)

df.to_csv('train_data_smt.txt', sep='\t', index=False)

In [50]:
# df_smt = pd.read_csv('train_data_smt.txt', sep='\t')

# X_train_smt = df_smt.drop('Churn', axis=1)  
# y_train_smt = df_smt['Churn']              